## Model Drift Detection Preparation

This notebook is used to create inference data for the drift detection notebooks.  The plan is to run these an hour or a day before so the workshop participants can use them for their training.

This notebook will have the bare minimum necessary for the training.  The rest will be part of the N4_drift_detection.ipynb notebook to actually demonstrate using assays for drift detection.

Because this is a preparation notebook, it is recommended that only the workspace/pipelines/model names are changed to execute this notebook.

## Steps

* Load the workspace, pipeline, and model versions.
* Perform sample inferences to:
  * Set the baseline
  * Perform "normal" inferences.
  * Perform inferences that should trigger alerts.

### Import Libraries

The first step will be to import our libraries, and set variables used through this tutorial.

In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

from IPython.display import display

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import datetime
import time


workspace_name = 'workshop-workspace-john-05'
main_pipeline_name = 'houseprice-estimator'
model_name_control = 'house-price-prime'

# ignoring warnings for demonstration
import warnings
warnings.filterwarnings('ignore')

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

### Retrieve Workspace, Pipeline, and Models

Retrieve the workspace, pipeline and model from notebook N1_deploy_a_model.ipynb.

In [3]:
# retrieve the previous workspace, model, and pipeline version

workspace_name = "workshop-workspace-john-cybersecurity"

workspace = wl.get_workspace(name=workspace_name)

# set your current workspace to the workspace that you just created
wl.set_current_workspace(workspace)

# optionally, examine your current workspace
wl.get_current_workspace()

model_name = 'aloha-prime'

prime_model_version = wl.get_model(model_name)

pipeline_name = 'aloha-fraud-detector'

pipeline = wl.get_pipeline(pipeline_name)

display(workspace)
display(prime_model_version)
display(pipeline)


{'name': 'workshop-workspace-john-cybersecurity', 'id': 14, 'archived': False, 'created_by': '76b893ff-5c30-4f01-bd9e-9579a20fc4ea', 'created_at': '2024-05-01T16:30:01.177583+00:00', 'models': [{'name': 'aloha-prime', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 5, 1, 16, 30, 43, 651533, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 5, 1, 16, 30, 43, 651533, tzinfo=tzutc())}, {'name': 'aloha-challenger', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 5, 1, 16, 38, 56, 600586, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 5, 1, 16, 38, 56, 600586, tzinfo=tzutc())}], 'pipelines': [{'name': 'aloha-fraud-detector', 'create_time': datetime.datetime(2024, 5, 1, 16, 30, 53, 995114, tzinfo=tzutc()), 'definition': '[]'}]}

Name,aloha-prime
Version,c719bc50-f83f-4c79-b4af-f66395a8da04
File Name,aloha-cnn-lstm.zip
SHA,fd998cd5e4964bbbb4f8d29d245a8ac67df81b62be767afbceb96a03d1a01520
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-01-May 16:30:43


name,aloha-fraud-detector
created,2024-05-01 16:30:53.995114+00:00
last_updated,2024-05-01 19:08:00.658437+00:00
deployed,True
arch,x86
accel,none
tags,
versions,"41b9260e-21a8-4f16-ad56-c6267d3bae93, 46bdd5a3-fc22-41b7-b1ea-8287c99c241e, 28f5443a-ff67-4f4f-bfc3-f3f95f3c6f83, 435b73f7-76a1-4514-bd41-2cb94d3e78ff, 551242a7-fe4c-4a61-a4c7-e7fcc97509dc, d22eb0d2-9cff-4c5f-a851-10b1a19d8c44, 262909e9-8779-4c56-a994-725ddd0b58c8, 4cdf8e11-1b9c-44ab-a16d-abb054b5e9fe, 6b3529b1-1ff1-454b-8896-460c8c90d667"
steps,aloha-prime
published,False


### Deploy Pipeline

Deploy the pipeline with the model version.

In [4]:
pipeline.clear()
pipeline.add_model_step(prime_model_version)

deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
pipeline.deploy(deployment_config=deploy_config)

name,aloha-fraud-detector
created,2024-05-01 16:30:53.995114+00:00
last_updated,2024-05-01 19:20:24.682678+00:00
deployed,True
arch,x86
accel,none
tags,
versions,"ea59aae6-1565-4321-9086-7f8dd2a8e1c2, 41b9260e-21a8-4f16-ad56-c6267d3bae93, 46bdd5a3-fc22-41b7-b1ea-8287c99c241e, 28f5443a-ff67-4f4f-bfc3-f3f95f3c6f83, 435b73f7-76a1-4514-bd41-2cb94d3e78ff, 551242a7-fe4c-4a61-a4c7-e7fcc97509dc, d22eb0d2-9cff-4c5f-a851-10b1a19d8c44, 262909e9-8779-4c56-a994-725ddd0b58c8, 4cdf8e11-1b9c-44ab-a16d-abb054b5e9fe, 6b3529b1-1ff1-454b-8896-460c8c90d667"
steps,aloha-prime
published,False


### Generate Sample Data

Before creating the assays, we must generate data for the assays to build from.

For this example, we will:

* Perform sample inferences based on lower priced houses and use that as our baseline.
* Generate inferences from specific set of high priced houses create inference outputs that will be outside the baseline.  This is used in later steps to demonstrate baseline comparison against assay analyses.

#### Inference Results History Generation

To start the demonstration, we'll create a baseline of values from houses with small estimated prices and set that as our baseline.

We will save the beginning and end periods of our baseline data to the variables `assay_baseline_start` and `assay_baseline_end`.

In [5]:
baseline_data = pd.read_json('../data/aloha_baseline.df.json')
assay_data = pd.read_json('../data/aloha_assay.df.json')

baseline_size = 500

# Where the baseline data will start
assay_baseline_start = datetime.datetime.now()

# These inputs will be random samples of small priced houses.  Around 30,000 is a good number
baseline_inference_set = baseline_data.sample(baseline_size, replace=True).reset_index(drop=True)

# Wait 60 seconds to set this data apart from the rest
time.sleep(60)
small_results = pipeline.infer(baseline_inference_set)

# Set the baseline end

assay_baseline_end = datetime.datetime.now()

#### Generate Numpy Baseline Values

This process generates a numpy array of the inference results used as baseline data in later steps.

In [8]:
# get the numpy values

# set the results to a non-array value
small_results_baseline_df = small_results.copy()
small_results_baseline_df['cryptolocker']=small_results['out.cryptolocker'].map(lambda x: x[0])
small_results_baseline_df

# set the numpy array
small_results_baseline = small_results_baseline_df['cryptolocker'].to_numpy()

#### Assay Test Data

The following will generate inference data for us to test against the assay baseline.  For this, we will add in house data that generate higher house prices than the baseline data we used earlier.

This process should take 6 minutes to generate the historical data we'll later use in our assays.  We store the DateTime `assay_window_start` to determine where to start out assay analyses.

In [9]:
# Get a spread of results

# # Set the start for our assay window period.
assay_window_start = datetime.datetime.now()

time.sleep(65)
inference_size = 1000

# Get a spread of values

baseline_inference_set = assay_data.sample(inference_size, replace=True).reset_index(drop=True)

pipeline.infer(baseline_inference_set)

time.sleep(65)

### Undeploy Main Pipeline

With the examples and tutorial complete, we will undeploy the main pipeline and return the resources back to the Wallaroo instance.

In [10]:
pipeline.undeploy()

name,aloha-fraud-detector
created,2024-05-01 16:30:53.995114+00:00
last_updated,2024-05-01 19:20:24.682678+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"ea59aae6-1565-4321-9086-7f8dd2a8e1c2, 41b9260e-21a8-4f16-ad56-c6267d3bae93, 46bdd5a3-fc22-41b7-b1ea-8287c99c241e, 28f5443a-ff67-4f4f-bfc3-f3f95f3c6f83, 435b73f7-76a1-4514-bd41-2cb94d3e78ff, 551242a7-fe4c-4a61-a4c7-e7fcc97509dc, d22eb0d2-9cff-4c5f-a851-10b1a19d8c44, 262909e9-8779-4c56-a994-725ddd0b58c8, 4cdf8e11-1b9c-44ab-a16d-abb054b5e9fe, 6b3529b1-1ff1-454b-8896-460c8c90d667"
steps,aloha-prime
published,False


### Store the Assay Values

We will store the following into a location configuration file:

* `small_results_baseline`:  Used to create the baseline from the numpy values from sample inferences.
* `assay_baseline_start`: When to start the baseline from the inference history.
* `assay_baseline_end`: When to end the baseline from the inference history.
* `assay_window_start`: When to start the assay window period for assay samples.

In [11]:
# skip this step if the file is already there

import numpy

numpy.save('./small_results_baseline.npy', small_results_baseline)

In [12]:
baseline_numpy = numpy.load('./small_results_baseline.npy')
baseline_numpy

array([1.38411885e-02, 6.98903700e-02, 1.72902760e-01, 1.01371060e-01,
       1.18515120e-02, 3.29993070e-02, 6.35678300e-02, 2.04431410e-02,
       5.54976280e-02, 3.33689700e-02, 1.63933440e-01, 5.01484130e-02,
       9.13416500e-02, 1.93939230e-02, 1.30328190e-02, 1.59948200e-01,
       1.50059490e-02, 1.80134130e-01, 3.50623620e-03, 1.64611250e-02,
       4.29595300e-02, 1.50780880e-02, 1.43549200e-01, 4.56962830e-03,
       4.97932730e-02, 1.64257660e-01, 7.37149900e-02, 1.10372186e-01,
       1.68252210e-02, 8.52306000e-02, 3.29359550e-02, 5.03419380e-02,
       1.25163240e-01, 2.78586650e-02, 6.16877850e-03, 3.07482650e-02,
       2.27204800e-02, 6.45729100e-02, 7.57607000e-02, 7.08777550e-03,
       2.97230650e-02, 3.59103600e-02, 2.30605320e-02, 3.27331540e-02,
       1.86745990e-01, 1.29913540e-02, 2.54665330e-02, 2.75201000e-02,
       7.10707060e-03, 1.50722850e-01, 9.01008800e-02, 8.99820400e-02,
       7.66398000e-04, 5.31407200e-02, 2.59052660e-02, 1.35844560e-01,
      

In [13]:
with open('./assay_baseline_start', 'w') as file:
    file.write(assay_baseline_start.strftime("%d-%b-%Y (%H:%M:%S.%f)"))
assay_baseline_start

datetime.datetime(2024, 5, 1, 13, 20, 32, 870761)

In [14]:
with open('./assay_baseline_end', 'w') as file:
    file.write(assay_baseline_end.strftime("%d-%b-%Y (%H:%M:%S.%f)"))
assay_baseline_end

datetime.datetime(2024, 5, 1, 13, 21, 34, 718837)

In [15]:
with open('./assay_window_start', 'w') as file:
    file.write(assay_window_start.strftime("%d-%b-%Y (%H:%M:%S.%f)"))
assay_window_start

datetime.datetime(2024, 5, 1, 13, 27, 1, 25108)

In [16]:
# read the assay baseline start datetime

with open('./assay_baseline_start', 'r') as file:
    assay_baseline_start_test = datetime.datetime.strptime(file.read(), "%d-%b-%Y (%H:%M:%S.%f)")
assay_baseline_start_test

datetime.datetime(2024, 5, 1, 13, 20, 32, 870761)

In [17]:
# read the assay baseline end datetime

with open('./assay_baseline_end', 'r') as file:
    assay_baseline_end_test = datetime.datetime.strptime(file.read(), "%d-%b-%Y (%H:%M:%S.%f)")
assay_baseline_end_test

datetime.datetime(2024, 5, 1, 13, 21, 34, 718837)

In [18]:
# read the assay window start datetime

with open('./assay_window_start', 'r') as file:
    assay_window_start_test = datetime.datetime.strptime(file.read(), "%d-%b-%Y (%H:%M:%S.%f)")
assay_window_start_test

datetime.datetime(2024, 5, 1, 13, 27, 1, 25108)